In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import cv2
from torch import nn
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import torchvision.transforms as tt
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchvision.models.video import r2plus1d_18, R2Plus1D_18_Weights
from sklearn.metrics import accuracy_score

In [13]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
torch.random.seed= seed_constant

BATCH_SIZE=5

# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 224, 224

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 9

# Specify the directory containing the UCF50 dataset.
DATASET_DIR = "/content/drive/MyDrive/Project Videos"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
# CLASSES_LIST = ["Diving", "Fencing", "Golf Swing", "Playing Guitar","High Jump",
#                 "Horse Riding", "Swing", "Punch","Playing Violin","Jump Rope",
#                 "Basketball","Shooting"]
CLASSES_LIST=['Person Walking','Person Running',"Person Exercising","Person Eating"]


## plotdf

In [14]:
# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (12,9),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }
CMAP = plt.cm.brg

plt.rcParams.update(params)

In [15]:
###-----------------------------------
### Function to plot Loss Curve
###-----------------------------------

def plot_tf_hist(hist_df):
    '''    Args:
      hist_df : pandas Dataframe with four columns
                For 'x' values, we will use index
    '''
    fig, axes = plt.subplots(1,2 , figsize = (15,6))

    # properties  matplotlib.patch.Patch
    props = dict(boxstyle='round', facecolor='aqua', alpha=0.4)
    facecolor = 'cyan'
    fontsize=12

    # Get columns by index to eliminate any column naming error
    y1 = hist_df.columns[0]
    y2 = hist_df.columns[1]
    y3 = hist_df.columns[2]
    y4 = hist_df.columns[3]

    # Where was min loss
    best = hist_df[hist_df[y3] == hist_df[y3].min()]

    ax = axes[0]

    hist_df.plot(y = [y1,y3], ax = ax, colormap=CMAP)


    # little beautification
    txtFmt = "{}: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(y1.capitalize(),
                           hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y3]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Min: {best[y3].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y3].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y3].to_numpy()[0]), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y1.capitalize())
    ax.set_title('Errors')
    ax.grid();
    ax.legend(loc = 'upper left') # model legend to upper left

    ax = axes[1]

    hist_df.plot( y = [y2, y4], ax = ax, colormap=CMAP)

    # little beautification
    txtFmt = "{}: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(y2.capitalize(),
                           hist_df.iloc[-1][y2],
                           hist_df.iloc[-1][y4]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.2, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Best: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y4].to_numpy()[0]), # location of text
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow


    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y2.capitalize())
    ax.grid()
    ax.legend(loc = 'lower left')

    plt.tight_layout()

## Preprocess

In [16]:
def frames_extraction(video_path,abn=False):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''

    # Declare a list to store video frames.
    frames_list = []

    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Find FPS
    FPS=video_reader.get(cv2.CAP_PROP_FPS)

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(3*FPS/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):
        if abn:
           # Set the current frame position of the video.
           video_reader.set(cv2.CAP_PROP_POS_FRAMES+int(3*FPS), frame_counter * skip_frames_window)
        else:
           video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        # Reading the frame from the video.
        success, frame = video_reader.read()

        # Check if Video frame is not successfully read then break the loop
        if not success:
            break

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)

    # Release the VideoCapture object.
    video_reader.release()

    # Return the frames list.
    return frames_list

In [17]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''

    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []

    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):

        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

        # Iterate through all the files present in the files list.
        counter=0
        abn=True if class_name=="Person Exercising" else False

        for file_name in files_list[:20]:
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path,abn)

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)

    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [18]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: Person Walking
Extracting Data of Class: Person Running
Extracting Data of Class: Person Exercising
Extracting Data of Class: Person Eating


In [19]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [20]:
len(features)

79

In [21]:
# transforms required by r2plus1d model
r21d_trans=tt.Compose([R2Plus1D_18_Weights.KINETICS400_V1.transforms()])

In [22]:
# functions to apply transforms to each frame
def apply_tansforms(feat):

  # list to store transformed frames
  feats=[]
  for i in range(len(feat)):

      #converting to array and reshaping in required format
      x=np.transpose(np.array(feat[i]), (0,3,1,2))
      # convertin to tensor to apply transforms
      a=torch.Tensor(x)
      # apply transforms and append to the list
      feats.append(r21d_trans(a))
  return feats

In [23]:
# getting list of frames in form of tensors
feats=apply_tansforms(features)

In [24]:
labels=torch.tensor(labels)
labels.shape

torch.Size([79])

In [25]:
# just verifying
len(feats)

79

In [303]:
# applying train test split
x_train,x_test,y_train,y_test=train_test_split(feats,labels,stratify=labels,test_size=0.25,shuffle=True,random_state=seed_constant)

In [304]:
len(x_train)

59

In [305]:
# checking device
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

Using cuda device


In [306]:
# creating classes for creating custom dataset
class ActivityDS(Dataset):
    def __init__(self, featuress,labelss, device = device):
        self.features = featuress
        self.device = device
        self.labels = labelss

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        # Extract features and label from the features and labels lists
        features = self.features[index]
        labels = self.labels[index]

        # put features and labels onto device
        features = features.to(device)
        labels = labels.to(device)

        return features, labels

In [307]:
# creating training andtesting datasets
train_ds = ActivityDS(x_train,y_train)
test_ds = ActivityDS(x_test,y_test)

In [308]:
# checking the shape of each frame
for i in x_train:
  print(i.shape)
  break

torch.Size([3, 9, 112, 112])


In [389]:
len(train_ds),len(test_ds)

(59, 20)

In [414]:
BATCH_SIZE=30

In [415]:
# Define data loaders for the training and testing datasets
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=20, shuffle=False)

In [416]:
# keeping default weights i.e. for Kinetics dataset
weights=R2Plus1D_18_Weights.DEFAULT
# Initializing the model
r21d=r2plus1d_18(weights=weights,progress=True)

In [417]:
from torchvision.models.video import r2plus1d_18, R2Plus1D_18_Weights
# keeping default weights i.e. for Kinetics dataset
weights=R2Plus1D_18_Weights.DEFAULT
# Initializing the model
r21d=r2plus1d_18(weights=weights,progress=True)
# chaning the output features to match our requirement
in_fc=r21d.fc.in_features
r21d.fc=nn.Linear(in_fc,4,bias=True)

# Freezing all weights except classifier weights and biases, So we will only be training classifier and not the main part
for param in r21d.parameters():
   param.requires_grad=False
r21d.fc.weight.requires_grad = True
r21d.fc.bias.requires_grad = True

# putting model on gpu
model=r21d.to(device)

# Defining optimizer, Loss function, Epochs for model fiting
EPOCHS=50
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),weight_decay=0.008) # only update parameters which are not freezed
loss_fn = nn.CrossEntropyLoss()

In [420]:
r21d.fc

Linear(in_features=512, out_features=4, bias=True)

In [423]:
# del model,optimizer

In [424]:
# putting model on gpu
model=r21d.to(device)

In [425]:

# Defining optimizer, Loss function, Epochs for model fiting
EPOCHS=50
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),weight_decay=0.008) # only update parameters which are not freezed
loss_fn = nn.CrossEntropyLoss()

In [426]:
# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss() # cross entropy loss

loss, tloss = [],[]
acc, tacc = [],[]
n_epoch = []

# Train the model for multiple epochs

for epoch in range(EPOCHS):

    # Training loop
    train_loss = 0.0
    train_acc = 0.0

    for i, data in enumerate(train_loader):

        inputs, targets = data

        optimizer.zero_grad()

        outputs = model(inputs)

        preds = torch.argmax(outputs, dim = 1)

        batch_loss = loss_fn(outputs, targets)
        batch_acc = accuracy_score(targets.cpu().numpy(), preds.cpu().numpy())

        batch_loss.backward()

        optimizer.step()

        train_loss += batch_loss.item() * inputs.size(0)
        train_acc += batch_acc * inputs.size(0)

    train_loss /= len(train_ds)
    train_acc /= len(train_ds)

    loss.append(train_loss)
    acc.append(train_acc)

    # Testing loop
    test_loss = 0.0
    test_acc = 0.0

    with torch.no_grad():

        for data in test_loader:

            inputs, targets = data

            outputs = model(inputs)

            preds = torch.argmax(outputs, dim = 1)

            batch_loss = loss_fn(outputs, targets)

            batch_acc = accuracy_score(targets.cpu().numpy(), preds.cpu().numpy())

            test_loss += batch_loss.item() * inputs.size(0)
            test_acc += batch_acc* inputs.size(0)

        test_loss /= len(test_ds)
        test_acc /= len(test_ds)

        tloss.append(test_loss)
        tacc.append(test_acc)
    n_epoch.append(epoch)

    print(f'At epoch {epoch} | Loss - train:{train_loss:.4f}, test:{test_loss:.4f} | Acc - train:{train_acc:.4f}, test:{test_acc:.4f}')

At epoch 0 | Loss - train:1.3720, test:1.2804 | Acc - train:0.2881, test:0.4000
At epoch 1 | Loss - train:1.2693, test:1.2444 | Acc - train:0.5254, test:0.4500
At epoch 2 | Loss - train:1.2011, test:1.2163 | Acc - train:0.5932, test:0.4500
At epoch 3 | Loss - train:1.1489, test:1.1614 | Acc - train:0.5763, test:0.5000
At epoch 4 | Loss - train:1.0498, test:1.1141 | Acc - train:0.7458, test:0.6000
At epoch 5 | Loss - train:0.9848, test:1.0802 | Acc - train:0.8305, test:0.6000
At epoch 6 | Loss - train:0.9104, test:1.0503 | Acc - train:0.8305, test:0.6500
At epoch 7 | Loss - train:0.8800, test:1.0199 | Acc - train:0.8475, test:0.7000
At epoch 8 | Loss - train:0.8142, test:0.9887 | Acc - train:0.9153, test:0.7500
At epoch 9 | Loss - train:0.7629, test:0.9588 | Acc - train:0.9492, test:0.6500
At epoch 10 | Loss - train:0.7140, test:0.9354 | Acc - train:0.9322, test:0.7500
At epoch 11 | Loss - train:0.6775, test:0.9132 | Acc - train:0.9492, test:0.7000
At epoch 12 | Loss - train:0.6876, tes

In [427]:
loss_df=pd.DataFrame({"loss":loss,"acc":acc,"val_loss":tloss,"val_acc":tacc})
loss_df

,loss,acc,val_loss,val_acc
0,1.372013,0.288136,1.280357,0.40
1,1.269326,0.525424,1.244354,0.45
2,1.201089,0.593220,1.216314,0.45
3,1.148921,0.576271,1.161415,0.50
4,1.049823,0.745763,1.114087,0.60
5,0.984806,0.830508,1.080170,0.60
6,0.910411,0.830508,1.050297,0.65
7,0.879961,0.847458,1.019923,0.70
8,0.814205,0.915254,0.988746,0.75
9,0.762935,0.949153,0.958785,0.65


In [428]:
loss_df[loss_df["val_loss"]==loss_df["val_loss"].min()]

,loss,acc,val_loss,val_acc
49,0.174493,1.0,0.67049,0.8


In [429]:
plot_tf_hist(loss_df)

In [430]:
def val_frames_extraction(video_path,SEQUENCE_LENGTH=9,TIME_SECODNS=3):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''

    # Declare a list to store video frames.

    frames_list = []
    vid_list=[]
    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)
    # Get Frame counts
    frame_count=video_reader.get(cv2.CAP_PROP_FRAME_COUNT)
    # Get FPS
    FPS=video_reader.get(cv2.CAP_PROP_FPS)
    # Find video length
    vid_len=frame_count/FPS
    # Finding frames in 3 seconds window
    thresh_frames=int(TIME_SECODNS*FPS)

    # Get the total number of frames in the video.
    # video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the the interval after which frames will be added to the list.
    # as we need 10 frames for each 3 second video
    skip_frames_window = max(int(thresh_frames/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    n_videos=int(vid_len/3)
    print(vid_len/3, f"So dividing into {n_videos} sub videos")
    for i in range(n_videos):
      count=i*SEQUENCE_LENGTH
      for frame_counter in range(SEQUENCE_LENGTH):
          # Set the current frame position of the video.
          video_reader.set(cv2.CAP_PROP_POS_FRAMES, (count+frame_counter) * skip_frames_window)

          # Reading the frame from the video.
          success, frame = video_reader.read()

          # Check if Video frame is not successfully read then break the loop
          if not success:
              break

          # Resize the Frame to fixed height and width.
          resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

          # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
          normalized_frame = resized_frame / 255

          # Append the normalized frame into the frames list
          frames_list.append(normalized_frame)

      vid_list.append(frames_list)
      frames_list =[]

    # Release the VideoCapture object.
    video_reader.release()

    # Return the frames list.
    return vid_list

In [433]:

def preprocess_pred_1(video_file_path):
    # video_file_path = "/content/run_test1.mp4"
    features=[]
    # Extract the frames of the video file.
    vid_list = val_frames_extraction(video_file_path)     

    # Converting the sub list to numpy arrays and apply transforms
    for i in range(len(vid_list)):
      vid_list[i]=np.asarray([vid_list[i]])
      vid_list[i]=apply_tansforms(vid_list[i])[0]

    # Return the frames, class index, and video file path.
    return vid_list

In [450]:
features_1=preprocess_pred_1("/content/exectest.mp4")

4.986111111111112 So dividing into 4 sub videos


In [451]:
with torch.no_grad():
    model.eval()
    y_preds=[]
    y_probas=[]
    for i in features_1:
      pred=model(i.unsqueeze(0).to(device))
      # print(pred[0])
      probas=(F.softmax(pred[0],dim=0))
      pred=torch.argmax(pred, dim = 1).to("cpu").numpy()
      y_preds.append(pred)
      y_probas.append(probas)

In [453]:
print(CLASSES_LIST)
class_predictions=[CLASSES_LIST[i[0]] for i in y_preds]
for i,j in zip(y_probas,class_predictions):
  print(i,j)

['Person Walking', 'Person Running', 'Person Exercising', 'Person Eating']
tensor([0.0172, 0.0253, 0.8566, 0.1009], device='cuda:0') Person Exercising
tensor([0.0078, 0.0249, 0.8835, 0.0838], device='cuda:0') Person Exercising
tensor([0.0174, 0.0169, 0.9020, 0.0637], device='cuda:0') Person Exercising
tensor([0.0046, 0.0198, 0.9387, 0.0370], device='cuda:0') Person Exercising


In [454]:
import joblib

In [456]:
model1=model.to("cpu")
joblib.dump(model1,"act_1.pkl")

['act_1.pkl']